# 7 - Functions

## 7.1. Writing Functions That Accept Any Number of Arguments

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))


In [2]:
avg(1, 2)

1.5

In [3]:
avg(1, 2, 3)

2.0

To accept any number of keyword arguments, use an argument that starts with **.

In [4]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
        name=name,
        attrs=attr_str,
        value=html.escape(value))
    return element


In [5]:
make_element('item', 'Albatross', size='large', quantity=6)

'<item size="large" quantity="6">Albatross</item>'

In [6]:
make_element('p', '<spam>')

'<p>&lt;spam&gt;</p>'

In [8]:
# generally
def anyargs(*args, **kwargs):
    print(args, type(args))     # tuple
    print(kwargs, type(kwargs)) # dict

anyargs("arg1", "arg2", kwarg1="kwarg1", kwarg2="kwarg2")

('arg1', 'arg2') <class 'tuple'>
{'kwarg1': 'kwarg1', 'kwarg2': 'kwarg2'} <class 'dict'>


## Writing Functions That Only Accept Keyword Arguments
Easy to implement if you place the keyword arguments after a * argument or a single unnamed *.

In [10]:
def recv(maxsize, *, block):
    print(block)

recv(1024, block=True)

True


In [11]:
recv(1024, True)

TypeError: recv() takes 1 positional argument but 2 were given

Keyword-only arguments are often a good way to enforce greater code clarity.

## Attaching Informational Metadata to Function Arguments
They are not type checks, nor do they make Python behave any differently than it did before. However, they might give useful hints to others reading the source code about what you had in mind.

In [12]:
def add(x:int, y:int) -> int:
    return x + y


In [13]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



In [14]:
add.__annotations__

{'x': int, 'y': int, 'return': int}

## Returning Multiple Values from a Function

In [15]:
def func():
    return 1, 2, 3

func()

(1, 2, 3)

## Defining Functions with Default Arguments

In [16]:
def func(a, b=3):
    return a + b


In [17]:
func(2)

5

In [18]:
func(3, 5)

8

## Defining Anonymous or Inline Functions

In [20]:
add = lambda x, y: x + y
add

<function __main__.<lambda>(x, y)>

In [21]:
add(1, 2)

3

## Capturing Variables in Anonymous Functions

In [22]:
x = 10
a = lambda y: x + y
a(10)

20

In [23]:
x = 20
b = lambda y: x + y
b(10)

30

In [24]:
a(10), b(10)

(30, 30)

The value of x in the lambda expressions is whatever the value of the x variable happens to be at the time of execution.

## Making an N-Argument Callable Work As a Callable with Fewer Arguments

In [1]:
def spam(a, b, c, d):
    print(a, b, c, d)


In [2]:
spam(1,2,3,4)

1 2 3 4


In [3]:
from functools import partial

s1 = partial(spam, 1)

In [4]:
s1(2,3,4)

1 2 3 4


In [5]:
s2 = partial(spam, d=42) 
s2(1,2,3)

1 2 3 42


## Replacing Single Method Classes with Functions
In many cases, single-method classes can be turned into functions using closures.

In [1]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    def increment(self):
        self.a += 1
        self.b += 1


In [2]:
myobj = MyClass(1, 2)
myobj.a, myobj.b

(1, 2)

In [3]:
myobj.increment()
myobj.a, myobj.b

(2, 3)

In [4]:
def myfunc(a, b):
    a = a
    b = b
    def increment():
        nonlocal a, b
        a = a + 1
        b = b + 1
        return a, b
    return increment


In [5]:
f = myfunc(1, 2)

In [6]:
f()

(2, 3)

In [7]:
f()

(3, 4)

In [8]:
f()

(4, 5)

##  Carrying Extra State with Callback Functions

In [9]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    # Invoke the callback with the result
    callback(result)


In [10]:
def print_result(result):
    print('Got:', result)

def add(x, y):
    return x + y


In [11]:
apply_async(add, (2, 3), callback=print_result)

Got: 5


In [12]:
apply_async(add, ('hello', 'world'), callback=print_result)

Got: helloworld


You could keep state info within an object.

In [13]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))


In [14]:
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)

[1] Got: 5


Or you could use a closure instead.

In [18]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler


In [19]:
handler = make_handler()
apply_async(add, (2, 3), callback=handler)

[1] Got: 5


## Inlining Callback Functions
Callback functions can be inlined into a function using generators and coroutines.

In [20]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)


In [21]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args


In [27]:
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
 
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper


In [28]:
def add(x, y):
    return x + y


In [29]:
@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')


In [30]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


## Accessing Variables Defined Inside a Closure

In [31]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n=', n)
 
    # Accessor methods for n
    def get_n():
        return n
    
    def set_n(value):
        nonlocal n
        n = value

    func.get_n = get_n
    func.set_n = set_n

    return func


In [32]:
f = sample()

In [33]:
f()

n= 0


In [34]:
f.set_n(10)

In [35]:
f()

n= 10


In [36]:
f.get_n()

10

***